In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [45]:
data = pd.read_csv('dados_correios.csv')

In [46]:
data.head()

,tipo_demanda,horario,local,volume_atendimentos,nota_avaliacao_ai,tempo_medio_atendimento,historico_demandas,estado,regiao,cidade,numero_funcionarios,mes,dia_semana,ano,volume_atendimentos_nao_finalizados,volume_atendimentos_finalizados
0,Cartas,22,Agência Central,457,1.35,10.51,180,SP,Sudeste,Salvador,27,12,Quinta,2013,7,450
1,Outro,3,Agência Bairro,102,4.13,19.46,221,MG,Sudeste,Salvador,39,2,Terça,2012,0,102
2,Expressa,17,Agência Bairro,113,1.62,11.03,669,MG,Nordeste,Belo Horizonte,37,4,Segunda,2010,0,113
3,Expressa,4,Centro de Distribuição,363,1.69,7.19,911,SP,Sudeste,São Paulo,39,7,Quinta,2010,80,283
4,Pacotes,15,Agência Central,423,4.32,18.27,732,RJ,Nordeste,São Paulo,26,2,Quinta,2023,13,410


In [47]:
data = data.drop('cidade', axis=1)

In [48]:
data = data.drop('regiao', axis=1)

In [49]:
data = data.drop('historico_demandas', axis=1)

## Engenharia de variaveis para modelo de previsão de demanda

### Média Diária, Mensal

In [50]:
import pandas as pd

# Calcular média diária e mensal
data['diaria_avg'] = data.groupby(['tipo_demanda', 'local', 'dia_semana'])['volume_atendimentos'].transform('mean')
data['mensal_avg'] = data.groupby(['tipo_demanda', 'local', 'mes'])['volume_atendimentos'].transform('mean')

# Calcular média móvel de 7 dias
data['media_movel_7dias'] = data.groupby(['tipo_demanda', 'local'])['volume_atendimentos'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# Média por local e tipo de demanda
data['media_atendimentos_por_local_demanda'] = data.groupby(['local', 'tipo_demanda'])['volume_atendimentos'].transform('mean')
data['media_atendimentos_por_local'] = data.groupby(['local'])['volume_atendimentos'].transform('mean')
data['media_atendimentos_por_demanda'] = data.groupby(['tipo_demanda'])['volume_atendimentos'].transform('mean')

In [51]:
data = data.dropna()

## Modelo

In [52]:
X = data.drop(['volume_atendimentos', 'volume_atendimentos_nao_finalizados', 'volume_atendimentos_finalizados'], axis=1)
y = data['volume_atendimentos']

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [53]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)
y_test = np.array(y_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE): 51.204707500000005
Mean Squared Error (MSE): 4689.078082975
Root Mean Squared Error (RMSE): 68.47684340691384
R-squared (R²): 0.7267958822656726


## Salvar o modelo

In [ ]:
import pickle
import gzip

with gzip.open('model_teste.pkl.gz', 'wb') as f:
    pickle.dump(model, f)

In [56]:
data.to_csv('Dados_tratados.csv', index = False)